In [1]:
import sys
sys.path.append(r'../')

In [2]:
from collections import Counter

from src.larp import LARP
from src.utils.utils_waterflow.local_search import local_search, erosion

from src.utils.utils_waterflow.clouds_generator import clouds_generator
from src.utils.get_data import random_data

In [3]:
k_vehicles = 2
Q_vehicle_capacity = 100
facility = 'F'

m_storages = 4
n_fields = 5

fields, storages, households, f, q, fs_dist, cs_dist, d = random_data(n_fields, m_storages)

In [4]:
larp = LARP(facility, 
            k_vehicles, 
            Q_vehicle_capacity, 
            fields, 
            storages, 
            households, 
            f, 
            q, 
            fs_dist, 
            cs_dist, 
            d)

# NOTE: Since it is difficult to find an optimal solution 
# trying with random X, Y and Z (decision variables)
# we fix a random X and set guroby to find and stop the very first feasible solution
larp.model.setParam('OutputFlag', 0)
larp.model.setParam('SolutionLimit', 1)
larp.build()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-12
-- LARP model build COMPLETED --


In [5]:
optimal_dows = dict()
P0_list = list()
UE_list = list()
E_list = list()

excluded_list = list()
discarded_list = list()

max_cloud = 3
max_pop = 10

min_ero = 2
max_UIE = 5

best_solution = None

In [6]:
clouds = clouds_generator(max_cloud, larp, max_pop)

for cloud in clouds:
    print('start to rain...')
    rainfall, discarded_dows = cloud.make_rain(E_list, discarded_list)
    print('stopped to rain.')
    discarded_list.extend(discarded_dows)

    print('gravity force push down dows...')
    for dow in rainfall:
        local_optimum, neighbours, excluded_dows, discarded_dows = local_search(larp, dow)
        excluded_list.extend(excluded_dows)
        discarded_list.extend(discarded_dows)
        UE_list.append(local_optimum) # for erosion process
    
        if local_optimum not in optimal_dows.keys():
            optimal_dows[local_optimum] = neighbours
    print('dows sink in some local positions.')

    for dow, neighbours in optimal_dows.items():
        print('dow:\n', dow)
        print('n. neighbours:', len(neighbours))
    
    dow_occurances = Counter(UE_list)
    dow_occurances = [dow for dow, occurs in dow_occurances.items() if occurs >= min_ero]
    print(f'{len(dow_occurances)} optimal(s) satisfy the erosion condition.')
    for dow in dow_occurances:
        
        neighbours = optimal_dows[dow]
        print('following dow can start the erosion process:\n', dow)
        print(f'dow has {len(neighbours)} neighbours that might be considered...')

        tmp = erosion(larp, dow, neighbours, max_UIE,
            excluded_list, discarded_list, optimal_dows, 
            P0_list, UE_list, E_list)
        
        dow_optimum, _, excluded_list, discarded_list, \
            optimal_dows, UE_list, E_list = tmp
        P0_list.append(dow_optimum)
        
        print('dow optimum:', dow_optimum)

start to rain...
stopped to rain.
gravity force push down dows...
dows sink in some local positions.
dow:
 objValue: 108892949.0
X: [1 1 0 0]
Y: [2 1 1 2 2]
Z: [0 1 0 2]

n. neighbours: 4
dow:
 objValue: 8889322256351.0
X: [1 1 1 0]
Y: [2 2 1 3 1]
Z: [0 2 0 3 1]

n. neighbours: 3
dow:
 objValue: 44466700148.0
X: [1 0 0 1]
Y: [1 1 4 1 1]
Z: [0 1 0 4]

n. neighbours: 4
1 optimal(s) satisfy the erosion condition.
following dow can start the erosion process:
 objValue: 108892949.0
X: [1 1 0 0]
Y: [2 1 1 2 2]
Z: [0 1 0 2]

dow has 4 neighbours that might be considered...
local solution is the local optimum.
search for the best neighbour of the local optimum...
no aligible neighbour is found.
local solution is the local optimum.
search for the best neighbour of the local optimum...
no aligible neighbour is found.
local solution is the local optimum.
search for the best neighbour of the local optimum...
no aligible neighbour is found.
local solution is the local optimum.
search for the best n

In [7]:
for i, dow in enumerate(P0_list):
    print(f'dow number {i}')
    print(dow)

dow number 0
objValue: 108892949.0
X: [1 1 0 0]
Y: [2 1 1 2 2]
Z: [0 1 0 2]

dow number 1
objValue: 18893087.5
X: [1 1 1 1]
Y: [2 1 4 2 2]
Z: [0 2 0 3 4 1]

dow number 2
objValue: 18893088.5
X: [1 1 1 1]
Y: [2 1 4 2 2]
Z: [0 2 0 4 1 3]

dow number 3
objValue: 18893088.5
X: [1 1 1 1]
Y: [2 1 4 2 2]
Z: [0 2 0 4 1 3]

dow number 4
objValue: 108892949.0
X: [1 1 0 0]
Y: [2 1 1 2 2]
Z: [0 1 0 2]

dow number 5
objValue: 44466700148.0
X: [1 0 0 1]
Y: [1 1 4 1 1]
Z: [0 1 0 4]

dow number 6
objValue: 44466700148.0
X: [1 0 0 1]
Y: [1 1 4 1 1]
Z: [0 1 0 4]

dow number 7
objValue: 108892949.0
X: [1 1 0 0]
Y: [2 1 1 2 2]
Z: [0 1 0 2]



In [8]:
if P0_list:
    obj_vals = [dow.obj_value for dow in P0_list]
    idx_min = obj_vals.index(min(obj_vals))
    best_solution = P0_list[idx_min]
    print(best_solution)

objValue: 18893087.5
X: [1 1 1 1]
Y: [2 1 4 2 2]
Z: [0 2 0 3 4 1]

